In [1]:
import pandas as pd

df = pd.read_parquet('hackathon_files_for_participants_ozon/test_data.parquet')

In [2]:
df.head()

,variantid,name,categories,color_parsed,pic_embeddings_resnet_v1,main_pic_embeddings_resnet_v1,name_bert_64,characteristic_attributes_mapping
0,51201254,Колодка TDM Electric четырехместная без заземл...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Сетевые...",[белый],"[[0.34383398, -0.2962618, 0.07987049, -0.08257...","[[0.38310742, -0.7876679, 0.5018278, 0.2090071...","[-0.5060825, 0.5773388, 0.59435517, -0.4958292...","{""Страна-изготовитель"":[""Китай""],""Бренд"":[""TDM..."
1,77151532,Клавиатура черная с черной рамкой для 25-011879,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Запчаст...",[черный],None,"[[0.50964713, 0.7958329, -1.4113188, 0.1999381...","[-0.43467724, 0.6614495, 0.48050267, -0.588880...","{""Страна-изготовитель"":[""Китай""],""Комплектация..."
2,89664856,"15.6"" Игровой ноутбук Acer Predator Helios 300...","{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Компьют...",[черный],"[[0.7804302, -0.245446, -0.67754817, -0.614691...","[[0.9958085, -0.113175124, -0.7623152, -0.9164...","[-0.70010763, 0.48152006, 0.47597092, -0.51727...","{""Видеокарта"":[""NVIDIA GeForce RTX 2070 (8 Гб)..."
3,90701982,Портативная колонка Borofone BR7 Empyreal Spor...,"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Акустик...","[red, красный]","[[-0.24636984, -1.0719914, -0.49986655, 0.3423...","[[-0.26596686, -1.143009, -0.5289628, 0.428558...","[-0.73135185, -0.039796613, 0.38907066, -0.496...","{""Основной материал корпуса"":[""Металл""],""Макси..."
4,92484118,Аккумулятор для Meizu BA712 ( M6s ),"{""1"": ""EPG"", ""2"": ""Электроника"", ""3"": ""Батарей...",None,None,"[[0.42047608, 0.75828516, 0.5440093, -0.006894...","[-0.600158, 0.13944691, 0.48706242, -0.5050975...","{""Рекомендовано для"":[""Meizu""],""Бренд"":[""Meizu..."


In [3]:
df_train_pairs = pd.read_parquet('hackathon_files_for_participants_ozon/test_pairs_wo_target.parquet')

In [4]:
df['cat3'] = df['categories'].apply(lambda x : eval(x)['3'])

In [11]:
df['color_parsed'] = df['color_parsed'].apply(lambda x: 'Colors: ' + ', '.join(x) if x is not None else '[UNK]')
df['text'] = df['name'] + ' ' + df['color_parsed']

In [12]:
import tqdm

dataset = []
for i in tqdm.tqdm(range(len(df_train_pairs))):
    t1,t2 = df_train_pairs.iloc[i].variantid1, df_train_pairs.iloc[i].variantid2
    name1 = df.loc[df.variantid == t1,'text'].values[0]
    name2 = df.loc[df.variantid == t2,'text'].values[0]
    dataset.append((name1, name2))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18084/18084 [00:08<00:00, 2116.20it/s]


In [7]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import tqdm
  
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
model1 = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", 
                                                           num_labels=2).cuda()
model1.load_state_dict(torch.load('colors_0_0.711_0.927.pth'))

model2 = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", 
                                                           num_labels=2).cuda()
model2.load_state_dict(torch.load('colors_1_0.714_0.926.pth'))
model3 = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", 
                                                           num_labels=2).cuda()
model3.load_state_dict(torch.load('colors_2_0.716_0.926.pth'))
model4 = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", 
                                                           num_labels=2).cuda()
model4.load_state_dict(torch.load('colors_3_0.727_0.93.pth'))
model5 = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", 
                                                           num_labels=2).cuda()
model5.load_state_dict(torch.load('colors_4_0.722_0.928.pth'))


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

<All keys matched successfully>

In [13]:
evaldf = []
ss = torch.nn.Softmax(dim=1)
for t in tqdm.tqdm(range(len(dataset))):
    name1, name2 = dataset[t]
    tks = tokenizer.encode_plus(name1[:600], name2[:600], max_length=512, pad_to_max_length=False,
                        return_attention_mask=True, return_tensors='pt', truncation=True)

    with torch.no_grad():
        score1 = ss(model1(tks['input_ids'].cuda(), 
                           attention_mask=tks['attention_mask'].cuda(),
                           token_type_ids=tks['token_type_ids'].cuda()).logits)[0][1].item()
        score2 = ss(model2(tks['input_ids'].cuda(), 
                           attention_mask=tks['attention_mask'].cuda(),
                           token_type_ids=tks['token_type_ids'].cuda()).logits)[0][1].item()
        score3 = ss(model3(tks['input_ids'].cuda(), 
                           attention_mask=tks['attention_mask'].cuda(),
                           token_type_ids=tks['token_type_ids'].cuda()).logits)[0][1].item()
        score4 = ss(model4(tks['input_ids'].cuda(), 
                           attention_mask=tks['attention_mask'].cuda(),
                           token_type_ids=tks['token_type_ids'].cuda()).logits)[0][1].item()
        score5 = ss(model5(tks['input_ids'].cuda(), 
                           attention_mask=tks['attention_mask'].cuda(),
                           token_type_ids=tks['token_type_ids'].cuda()).logits)[0][1].item()

        score = (score1 + score2 + score3 + score4 + score5) * 0.2
        
        #score = score1
        evaldf.append((df_train_pairs.iloc[t].variantid1,
                       df_train_pairs.iloc[t].variantid2,
                       score))
        
evaldf = pd.DataFrame(evaldf)
evaldf.columns = ["variantid1","variantid2","colorbert"]

nn_features = pd.read_parquet('nn/test.parquet')
nn_features = nn_features.merge(evaldf, on=["variantid1","variantid2"], how='left')
nn_features.to_parquet('nn/test.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18084/18084 [12:23<00:00, 24.34it/s]


In [14]:
evaldf.to_parquet('test_colorbert.parquet')